# 1) A) Loading Data for Zip codes in Bournemouth

In [7]:
import pandas as pd
import requests

In [8]:
!wget -O 'temp.zip' https://www.getthedata.com/downloads/open_postcode_geo.csv.zip    

--2020-07-25 22:15:11--  https://www.getthedata.com/downloads/open_postcode_geo.csv.zip
Resolving www.getthedata.com (www.getthedata.com)... 95.154.195.188
Connecting to www.getthedata.com (www.getthedata.com)|95.154.195.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64607200 (62M) [application/zip]
Saving to: ‘temp.zip’

100%[======================================>] 64,607,200   107MB/s   in 0.6s   

2020-07-25 22:15:12 (107 MB/s) - ‘temp.zip’ saved [64607200/64607200]



In [9]:
!unzip temp.zip
!rm temp.zip 

Archive:  temp.zip
  inflating: open_postcode_geo.csv   
  inflating: readme.txt              
  inflating: licence.txt             


In [10]:
#import os
#for subdir, dirs, files in os.walk('./'):
#    for file in files:
#      print(file)

In [11]:
df1_zip = pd.read_csv('open_postcode_geo.csv', names=["postcode","status","usertype","easting","northing","positional_quality_indicator","country","latitude","longitude","postcode_no_space","postcode_fixed_width_seven","postcode_fixed_width_eight","postcode_area","postcode_district","postcode_sector","outcode","incode"])
#df1_zip.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df_bournemouth_active = df1_zip.loc[(df1_zip['status'] == 'live') & (df1_zip['country'] == 'England') & (df1_zip['postcode_area']=='BH')]
#[[df1_zip['status'] == 'live'] && [df1_zip['country'] == 'England']

In [13]:
df_bournemouth_active.head()

,postcode,status,usertype,easting,northing,positional_quality_indicator,country,latitude,longitude,postcode_no_space,postcode_fixed_width_seven,postcode_fixed_width_eight,postcode_area,postcode_district,postcode_sector,outcode,incode
173266,BH1 1AA,live,large,409722,92041,1,England,50.727909,-1.863623,BH11AA,BH1 1AA,BH1 1AA,BH,BH1,BH1 1,BH1,1AA
173268,BH1 1AF,live,small,409181,91523,1,England,50.723260,-1.871300,BH11AF,BH1 1AF,BH1 1AF,BH,BH1,BH1 1,BH1,1AF
173270,BH1 1AH,live,small,409265,91464,1,England,50.722728,-1.870112,BH11AH,BH1 1AH,BH1 1AH,BH,BH1,BH1 1,BH1,1AH
173271,BH1 1AJ,live,small,409162,92265,1,England,50.729932,-1.871551,BH11AJ,BH1 1AJ,BH1 1AJ,BH,BH1,BH1 1,BH1,1AJ
173272,BH1 1AL,live,small,409284,91366,1,England,50.721846,-1.869845,BH11AL,BH1 1AL,BH1 1AL,BH,BH1,BH1 1,BH1,1AL


In [14]:
#df_bournemouth_active.sort_values(by='outcode').groupby('outcode')['postcode'].count().plot(kind='bar', figsize =(10,10))
df_bournemouth_active[["latitude", "longitude"]] = df_bournemouth_active[["latitude", "longitude"]].apply(pd.to_numeric)

df_bournemouth_active.dtypes

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


postcode                         object
status                           object
usertype                         object
easting                          object
northing                         object
positional_quality_indicator      int64
country                          object
latitude                        float64
longitude                       float64
postcode_no_space                object
postcode_fixed_width_seven       object
postcode_fixed_width_eight       object
postcode_area                    object
postcode_district                object
postcode_sector                  object
outcode                          object
incode                           object
dtype: object

In [15]:
df_zip_group = df_bournemouth_active.groupby('outcode')['latitude','longitude'].mean()
df_zip_group = df_zip_group.drop(['BH20','BH21','BH22','BH23','BH24','BH25'])


In [16]:
df_zip_group.reset_index(inplace=True)

#df_zip_group

In [17]:
df_zip_group.columns = ['Postal Code','Latitude', 'Longitude']

In [18]:
df_zip_group['House Price Detached'] = 0.0
df_zip_group['House Price Flats'] = 0.0

df_zip_group[["House Price Detached", "House Price Flats"]] = df_zip_group[["House Price Detached", "House Price Flats"]].apply(pd.to_numeric)

df_zip_group.dtypes

Postal Code              object
Latitude                float64
Longitude               float64
House Price Detached    float64
House Price Flats       float64
dtype: object

In [19]:
for ind in df_zip_group.index:
    #print(df_zip_group['latitude'][outcode])
    df_prices = pd.read_html('https://www.streetlist.co.uk/bh/'+df_zip_group['Postal Code'][ind])
    df_prices = df_prices[0]
    #print(df_prices['Type of Property'])
   # print(' '+str(df_prices['Type of Property'][3]) +' '+ str(df_prices['2018 Average'][0]))

    df_zip_group['House Price Detached'][ind] = float(df_prices['2018 Average'][0].replace('£','').replace(',',''))
    df_zip_group['House Price Flats'][ind] = float(df_prices['2018 Average'][3].replace('£','').replace(',',''))
    print(type(df_zip_group['House Price Flats'][ind]))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [80]:
df_post_names = pd.read_html('https://en.wikipedia.org/wiki/BH_postcode_area')
df_post_names = df_post_names[1]
df_post_names.columns = ['Postal Code','Post Town','Coverage','Authority']
df_post_names = df_post_names.drop(['Authority'],axis=1)

In [20]:
#df_zip_group.head()



df_final = df_zip_group


#print('Before dropping Boroughs ='.join(df.shape))
#df_wikipedia= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

#there are no Postal codes repeated so we dont need to concatinate the Neighbourhoods.

#df_wikipedia.shape

#df_zip_group.dtypes
#df_final[["House Price Detached", "House Price Flats"]]
df_zip_group.dtypes


Postal Code              object
Latitude                float64
Longitude               float64
House Price Detached    float64
House Price Flats       float64
dtype: object

In [21]:
import requests
from pandas.io.json import json_normalize


fourSqAPIURL = "https://api.foursquare.com/v2/venues/explore"
CLIENT_ID ="NINRXL4F12KA0T5Y40SQD1CGLLM2FK1XGE1YZBYMVQINX45V"
CLIENT_SECRET= "4NSTMXF2D5VWNSAMNRUMP3RC1W4NHOLDSVOLK3UX1BXBAZD5"


In [22]:
def fetch_venue_Details(postalcode,latitude,longitude,v_list):
    new_venue = {'Postal Code':postalcode,'Latitude':latitude, 'Longitude':longitude,'Type':[],'Name':[]}

    new_venue["Name"] = v_list["venue"]["name"]
    new_venue["Type"] = v_list["venue"]["categories"][0]['pluralName']
    

    new_venues = pd.DataFrame(new_venue, index =[0])
    
  
   # new_venues = new_venues.append(new_venue,ignore_index=True)
    #print('after--- '+ str(new_venues))
    return new_venues

In [23]:
all_venues= pd.DataFrame()


for ind in df_final.index:

   
    lat = str(df_final['Latitude'][ind])
    lng = str(df_final['Longitude'][ind])

    #print(str("\n"+ df_final['outcode'][ind])+ "+++++++++++"+ str(ind)+"\n")
 
    payload = {'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET, 'll': lat+','+lng , 'radius':1500, 'v':'20180323'}

    resp = requests.get(fourSqAPIURL,params=payload)

    data = resp.json()
    
    #print(data)
    try:
        for items in data["response"]["groups"][0]["items"]:
           
            all_venues = all_venues.append(fetch_venue_Details(df_final['Postal Code'][ind],lat,lng,items))
            #print('reached')
            all_venues.head()
    except:
        print(str(df_final['outcode'][ind])+ "-----EMPTY------------")
        empty_venue = {'Postal Code':df_final['Postal Code'][ind],'Latitude':lat, 'Longitude':lng,'Type':[],'Name':[]}
        all_venues.append(pd.DataFrame(empty_venue))

In [24]:
all_venues = all_venues.reset_index(drop=True)


all_venues.head()

,Postal Code,Latitude,Longitude,Type,Name
0,BH1,50.724864264209,-1.8609877127496226,Pizza Places,Bournemouth Pizza Co.
1,BH1,50.724864264209,-1.8609877127496226,Italian Restaurants,La Piccola Italia
2,BH1,50.724864264209,-1.8609877127496226,Beaches,Bournemouth Beach (Toft Zig Zag)
3,BH1,50.724864264209,-1.8609877127496226,Indian Restaurants,Spice of Lahore
4,BH1,50.724864264209,-1.8609877127496226,Pubs,Inferno


In [25]:
neighbourhood_analysis = pd.get_dummies(all_venues,prefix='',prefix_sep='',columns=['Type'])

In [26]:
mean_data = neighbourhood_analysis.set_index(['Postal Code'])
mean_data = mean_data.groupby(level='Postal Code').mean()

In [27]:
mean_data.head()

,American Restaurants,Art Museums,Arts & Crafts Stores,Athletics & Sports,Bakeries,Bars,Beaches,Bed & Breakfasts,Beer Bars,Bistros,...,Tennis Stadiums,Thai Restaurants,Theaters,Trails,Train Stations,Turkish Restaurants,Vegetarian / Vegan Restaurants,Video Game Stores,Warehouse Stores,Women's Stores
Postal Code,,,,,,,,,,,,,,,,,,,,,
BH1,0.0,0.033333,0.0,0.0,0.0,0.000000,0.100000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.000000
BH10,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.071429
BH11,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
BH12,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.045455,0.000000,0.0,0.0,0.0,0.000000
BH13,0.0,0.000000,0.0,0.0,0.0,0.066667,0.266667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000


# For Segmenting we will segment into 4 distinct clusters

In [28]:
from sklearn.cluster import KMeans

#clusters
k=3

kmean_attractions = KMeans(n_clusters = k, random_state = 0, max_iter =500).fit(mean_data.to_numpy())
kmean_attractions.labels_

kmean_housing_prices = KMeans(n_clusters = k, random_state = 0, max_iter =500).fit(df_final[['House Price Detached','House Price Flats']].to_numpy())
kmean_housing_prices.labels_



array([1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
      dtype=int32)

In [29]:
mean_data['Attractions_Segment'] = kmean_attractions.labels_

data_analysis = mean_data.groupby('Attractions_Segment').mean()

data_analysis

,American Restaurants,Art Museums,Arts & Crafts Stores,Athletics & Sports,Bakeries,Bars,Beaches,Bed & Breakfasts,Beer Bars,Bistros,...,Tennis Stadiums,Thai Restaurants,Theaters,Trails,Train Stations,Turkish Restaurants,Vegetarian / Vegan Restaurants,Video Game Stores,Warehouse Stores,Women's Stores
Attractions_Segment,,,,,,,,,,,,,,,,,,,,,
0,0.006061,0.00303,0.00303,0.00303,0.003030,0.036364,0.052273,0.003788,0.009091,0.003788,...,0.00303,0.015152,0.006818,0.00303,0.003788,0.018182,0.006061,0.012121,0.000000,0.003030
1,0.000000,0.00000,0.00000,0.00000,0.014286,0.030952,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.004167,0.000000,0.00000,0.005682,0.000000,0.000000,0.000000,0.005952,0.008929
2,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
cols = list(data_analysis.columns)
indices = list(data_analysis.index)


for ind in indices:
    print ('\n\n----------------------------SEGMENT =' + str(ind+1)+'-------------------------------------------------------------------------------------------------------')
    for col in cols:
        if(data_analysis[col][ind]>0.040):
            print(str(col)+'='+str(data_analysis[col][ind]))



----------------------------SEGMENT =1-------------------------------------------------------------------------------------------------------
Beaches=0.052272727272727276
Cafés=0.06818181818181818
Coffee Shops=0.10984848484848485
Parks=0.05454545454545454
Pubs=0.06969696969696969


----------------------------SEGMENT =2-------------------------------------------------------------------------------------------------------
Coffee Shops=0.0625
Furniture / Home Stores=0.042316017316017314
Grocery Stores=0.19356060606060604
Pubs=0.061607142857142846
Supermarkets=0.0774621212121212


----------------------------SEGMENT =3-------------------------------------------------------------------------------------------------------
Coffee Shops=0.16666666666666666
Construction & Landscaping=0.3333333333333333
Fish & Chips Shops=0.16666666666666666
Gastropubs=0.16666666666666666
Pubs=0.16666666666666666


In [31]:
mean_data['Housing_Segment'] = kmean_housing_prices.labels_

In [32]:
df_merged = pd.merge(df_final,mean_data[['Attractions_Segment','Housing_Segment']], how='left', on='Postal Code')

Attractions_Segment
0    11
1     8
2     1
Name: Postal Code, dtype: int64

In [81]:
df_merged = pd.merge(df_merged,df_post_names,how='left',on='Postal Code')

,Postal Code,Latitude,Longitude,House Price Detached,House Price Flats,Attractions_Segment,Housing_Segment,Post Town,Coverage
3,BH12,50.737,-1.928,312703.000,165664.000,1,1,POOLE,"Branksome, Alderney, Upper Parkstone, Newtown,..."
2,BH11,50.762,-1.920,314385.000,156196.000,1,1,BOURNEMOUTH,"Kinson, Bear Cross, West Howe, Wallisdown, Bea..."
1,BH10,50.760,-1.893,319981.000,161868.000,1,1,BOURNEMOUTH,"Kinson, East Howe, Northbourne, Redhill, Ensbu..."
8,BH17,50.747,-1.980,332855.000,145224.000,1,1,POOLE,"Canford Heath, Creekmoor"
19,BH9,50.750,-1.873,351483.000,174347.000,1,1,BOURNEMOUTH,"Winton, Moordown, Throop, Muscliff"
6,BH15,50.722,-1.985,363262.000,222399.000,0,1,POOLE,"Poole town centre, Hamworthy, Longfleet, Oakda..."
7,BH16,50.740,-2.048,388158.000,164114.000,2,1,POOLE,"Upton, Turlin Moor, Lytchett Minster, Lytchett..."
13,BH31,50.876,-1.873,411698.000,195933.000,1,1,VERWOOD,Verwood
18,BH8,50.742,-1.852,421272.000,172461.000,0,1,BOURNEMOUTH,"Bournemouth railway station, Malmesbury Park, ..."
17,BH7,50.736,-1.827,437582.000,160541.000,0,1,BOURNEMOUTH,"Littledown, Iford"


In [34]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df_merged.groupby('Housing_Segment').mean()[['House Price Detached','House Price Flats']]

,House Price Detached,House Price Flats
Housing_Segment,,
0,640118.667,231494.833
1,392803.154,185996.692
2,1612349.000,456231.000


In [105]:
!pip install folium
import folium
 
def pricing_string(indicator):
    if indicator == 0:
        return '$$'
    elif indicator == 1:
        return'$'
    elif indicator == 2:
        return '$$$'
    
def housing_string(indicator):
    if indicator == 0:
        return 'Beach Town'
    elif indicator == 1:
        return'Residential'
    elif indicator == 2:
        return 'Nature'
    
m = folium.Map(location=[50.720806,-1.904755],zoom_start=12.5)

for ind in df_merged.index:

    if(df_merged['Attractions_Segment'][ind] == 0):
        div = folium.DivIcon(html=(
            '<svg>'
              '<g>'
                '<rect x="0" y="0" width="80" height="30" fill="green" stroke="green" stroke-width="3"></rect>'
                 '<text x="5" y="11" font-family="Verdana" font-size="12" fill="white">'+str(df_merged['Postal Code'][ind])+' '+pricing_string(df_merged['Housing_Segment'][ind])+'</text>'
                 '<text x="5" y="25" font-family="Verdana" font-size="12" fill="white">Beach Town</text>'
              '</g>'
            '</svg>'
        ))
    elif(df_merged['Attractions_Segment'][ind] == 1):
         div = folium.DivIcon(html=(
            '<svg>'
              '<g>'
                '<rect x="0" y="0" width="80" height="30" fill="magenta" stroke="magenta" stroke-width="3"></rect>'
                 '<text x="5" y="11" font-family="Verdana" font-size="12" fill="white">'+str(df_merged['Postal Code'][ind])+' '+pricing_string(df_merged['Housing_Segment'][ind])+'</text>'
                 '<text x="5" y="25" font-family="Verdana" font-size="12" fill="white">Residential</text>'
               #  '<tspan="0" y="0">Hii</tspan>'
               #  '<tspan="7" y="7">'+str(df_merged['Postal Code'][ind])+'</tspan>'             
                 
              '</g>'
            '</svg>'
            ))
    elif(df_merged['Attractions_Segment'][ind] == 2):
         div = folium.DivIcon(html=(
            '<svg>'
              '<g>'
                '<rect x="0" y="0" width="80" height="30" fill="blue" stroke="blue" stroke-width="3"></rect>'
               '<text x="5" y="11" font-family="Verdana" font-size="12" fill="white">'+str(df_merged['Postal Code'][ind])+' '+pricing_string(df_merged['Housing_Segment'][ind])+'</text>'
                 '<text x="5" y="25" font-family="Verdana" font-size="12" fill="white">Nature</text>'
              '</g>'
            '</svg>'
            ))
    folium.Marker((df_merged['Latitude'][ind],df_merged['Longitude'][ind]), icon=div).add_to(m)

m

In [108]:
df_formatted_merged = df_merged
column_names = ["Postal Code","Post Town","Coverage","Latitude","Longitude","Attractions_Segment","House Price Detached","House Price Flats","Housing_Segment"]

df_formatted_merged = df_formatted_merged.reindex(columns=column_names)


df_formatted_merged['Housing_Segment'] = df_formatted_merged.apply(lambda row : pricing_string(row['Housing_Segment']),axis = 1)

df_formatted_merged.sort_values(by='House Price Detached')

df_formatted_merged['Attractions_Segment'] = df_formatted_merged.apply(lambda row : housing_string(row['Attractions_Segment']),axis = 1)
#df_formatted_merged['Attractions_Segment'] = df_formatted_merged.apply(lambda row : if row['Attractions_Segment'] == 1,axis = 1)
#df_formatted_merged['Attractions_Segment'] = df_formatted_merged.apply(lambda row : if row['Attractions_Segment'] == 0,axis = 1)

df_formatted_merged.sort_values(by='House Price Detached',ascending = False)

,Postal Code,Post Town,Coverage,Latitude,Longitude,Attractions_Segment,House Price Detached,House Price Flats,Housing_Segment
4,BH13,POOLE,"Canford Cliffs, Sandbanks, Branksome Park, Bro...",50.709,-1.921,Beach Town,1612349.000,456231.000,$$$
12,BH3,BOURNEMOUTH,"Talbot Woods, Winton south",50.737,-1.883,Beach Town,718151.000,225350.000,$$
14,BH4,BOURNEMOUTH,"Westbourne, Branksome Woods",50.721,-1.900,Beach Town,714105.000,244530.000,$$
5,BH14,POOLE,"Lower Parkstone, Lilliput, Penn Hill",50.721,-1.944,Residential,712123.000,290629.000,$$
11,BH2,BOURNEMOUTH,Bournemouth town centre (West Cliff),50.721,-1.884,Beach Town,594750.000,188105.000,$$
15,BH5,BOURNEMOUTH,"Boscombe, Pokesdown",50.725,-1.832,Beach Town,556806.000,215850.000,$$
10,BH19,SWANAGE,"Swanage, Studland, Langton Matravers, Worth Ma...",50.612,-1.971,Beach Town,544777.000,224505.000,$$
16,BH6,BOURNEMOUTH,"Southbourne, Tuckton, Wick",50.728,-1.802,Beach Town,493807.000,223882.000,$
9,BH18,BROADSTONE,Broadstone,50.760,-1.998,Residential,485518.000,293388.000,$
0,BH1,BOURNEMOUTH,"Bournemouth town centre (East Cliff), Springbo...",50.725,-1.861,Beach Town,473737.000,181940.000,$
